In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from datetime import datetime
from tqdm import tqdm
import json

class WebPageExtractor:
    def __init__(self, url):
        self.url = url
        self.soup = None
        self.json_data = None

    def fetch_page(self):
        """Fetches the web page content and initializes BeautifulSoup."""
        try:
            response = requests.get(self.url)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            self.soup = BeautifulSoup(response.content, 'html.parser')
            print("Page fetched successfully.")
        except requests.RequestException as e:
            print(f"Failed to fetch page: {e}")
            self.soup = None

    def extract_json_metadata(self):
        """Extracts JSON metadata from the <script> tag with type 'application/ld+json'."""
        if self.soup:
            script_tag = self.soup.find('script', type='application/ld+json')
            if script_tag:
                try:
                    self.json_data = json.loads(script_tag.string)
                    print("JSON metadata extracted successfully.")
                except json.JSONDecodeError as e:
                    print(f"Failed to parse JSON metadata: {e}")
                    self.json_data = None
            else:
                print("No JSON metadata found.")
        else:
            print("Soup object not initialized. Call fetch_page() first.")

    def extract_author(self):
        """Extracts author(s) from the JSON metadata."""
        if self.json_data:
            author_info = self.json_data.get("author", None)
            if author_info:
                if isinstance(author_info, list):
                    authors = [author.get("name", "Unknown") for author in author_info]
                else:
                    authors = [author_info.get("name", "Unknown")]
                return authors
            else:
                return "No author information found."
        return "No JSON metadata found."

    def extract_date(self):
        """Extracts the publication date from the JSON metadata."""
        if self.json_data:
            return self.json_data.get("datePublished", "No publication date found.")
        return "No JSON metadata found."

    def extract_headline(self):
        """Extracts the headline from the JSON metadata."""
        if self.json_data:
            return self.json_data.get("headline", "No headline found.")
        return "No JSON metadata found."

    def extract_content(self):
        """Extracts the article content from <p> tags."""
        if self.soup:
            paragraphs = self.soup.find_all('p')
            article_content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
            return article_content if article_content else "No content found."
        return "Soup object not initialized. Call fetch_page() first."

class RSSFeedExtractor:
    def __init__(self, rss_urls):
        self.rss_urls = rss_urls
        self.df = None

    def get_source_name(self, url):
        domain = urlparse(url).netloc
        source = domain.replace('www.', '').split('.')[0]
        return source

    def parser_items_rss(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'xml')
        items = soup.find_all('item')
        source = self.get_source_name(url)

        lista_items = []
        for item in items:
            row = {
                "title": item.find('title').text if item.find('title') else None,
                "description": item.find('description').text if item.find('description') else None,
                "url": item.find('link').text if item.find('link') else None,
                "pubDate": item.find('pubDate').text if item.find('pubDate') else None,
                "source": source
            }
            lista_items.append(row)

        return lista_items

    def fetch_rss_feeds(self):
        datos = []
        for url in self.rss_urls:
            datos.extend(self.parser_items_rss(url))
        
        self.df = pd.DataFrame(datos)
        self.df = self.df.drop(['pubDate'], axis=1)
        self.df["type"] = self.df["url"].str.extract(r"https://www\.bbc\.com/news/([^/]+)/")
        self.df = self.df[self.df['type'] == 'articles']
        return self.df

class WebPageMetadataExtractor:
    def __init__(self, df):
        self.df = df

    def extract_webpage_info(self, row):
        url = row['url']
        extractor = WebPageExtractor(url)
        
        # Fetch page and extract metadata
        try:
            extractor.fetch_page()
            extractor.extract_json_metadata()
            return {
                'Author': extractor.extract_author(),
                'Date Published': extractor.extract_date(),
                'Headline': extractor.extract_headline(),
                'Content': extractor.extract_content(),
            }
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            return {
                'Author': None,
                'Date Published': None,
                'Headline': None,
                'Content': None,
            }

    def fetch_webpage_metadata(self):
        # Use tqdm for progress bar during extraction
        tqdm.pandas()
        extracted_data = self.df.progress_apply(self.extract_webpage_info, axis=1, result_type='expand')
        return pd.concat([self.df, extracted_data], axis=1)

class FilteredArticles:
    def __init__(self, rss_urls):
        self.rss_extractor = RSSFeedExtractor(rss_urls)
        self.metadata_extractor = None
        self.df = None

    def fetch_rss_articles(self):
        # Fetch RSS feeds and parse items
        self.df = self.rss_extractor.fetch_rss_feeds()
        return self.df

    def fetch_webpage_metadata(self):
        # Fetch metadata for the articles fetched from RSS
        self.metadata_extractor = WebPageMetadataExtractor(self.df)
        self.df = self.metadata_extractor.fetch_webpage_metadata()
        return self.df

    def filter_by_date(self):
        # Convert the "Date Published" column to datetime
        self.df['Date Published'] = pd.to_datetime(self.df['Date Published'])

        # Get today's date
        today = datetime.now().date()

        # Filter articles published today
        filtered_df = self.df[self.df['Date Published'].dt.date == today]
        return filtered_df


In [ ]:
# RSS feed URLs
world_news = [
    'https://feeds.bbci.co.uk/news/business/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/education/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/health/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/technology/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/world/rss.xml?edition=uk',
    'https://feeds.bbci.co.uk/news/science_and_environment/rss.xml?edition=uk'
]

# Create the main class instance
filtered_articles = FilteredArticles(world_news)

# Fetch RSS articles
df_articles = filtered_articles.fetch_rss_articles()

# Fetch webpage metadata
df_metadata = filtered_articles.fetch_webpage_metadata()

# Filter articles published today
df_filtered = filtered_articles.filter_by_date()

  1%|          | 2/201 [00:00<00:15, 13.03it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


  2%|▏         | 4/201 [00:00<00:19, 10.08it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


  3%|▎         | 7/201 [00:00<00:24,  7.92it/s]

...

JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


 99%|█████████▉| 199/201 [00:22<00:00,  8.46it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


100%|██████████| 201/201 [00:22<00:00,  9.00it/s]

Page fetched successfully.
JSON metadata extracted successfully.
Page fetched successfully.
JSON metadata extracted successfully.


In [8]:
# Assuming `df_filtered` is your filtered DataFrame
df_filtered = df_filtered.reset_index(drop=True)

In [10]:
df_filtered

,title,description,url,source,type,Author,Date Published,Headline,Content
0,Real pay rises at fastest rate since 2021,Wage growth increased by 3.4% after taking int...,https://www.bbc.com/news/articles/c4g372w32vjo,feeds,articles,[Michael Race],2025-01-21 07:11:07.216000+00:00,Pay rises at fastest rate for more than three ...,Real pay in the UK has risen at its fastest ra...
1,Delay to TikTok ban gets Trump sign-off,The order was among a slew of directives Trump...,https://www.bbc.com/news/articles/cd0j24rj4ryo,feeds,articles,[Lily Jamali and Peter Hoskins],2025-01-21 02:12:14.130000+00:00,President Trump signs executive order delaying...,President Trump has signed an executive order ...
2,Stock markets cautious as Trump signals new ta...,The US president said he is considering imposi...,https://www.bbc.com/news/articles/c1ezgdj7wvpo,feeds,articles,[João da Silva],2025-01-21 00:49:46.632000+00:00,Stock markets cautious as Trump signals new ta...,Stock markets in the Asia-Pacific region made ...
3,Inside Iceland's futuristic farm growing algae...,Next to a geothermal plant in Iceland a start-...,https://www.bbc.com/news/articles/c4gjry6dv4yo,feeds,articles,[Adrienne Murray],2025-01-21 00:00:04.073000+00:00,Inside Iceland's futuristic farm growing algae...,In the shadow of Iceland’s largest geothermal ...
4,Whitesnake guitarist John Sykes dies at 65,"A statement said Sykes, who co-wrote some of t...",https://www.bbc.com/news/articles/cp8qvz0gq90o,feeds,articles,[Christy Cooney],2025-01-21 04:40:41.637000+00:00,Whitesnake and Thin Lizzy guitarist John Sykes...,"The British rock guitarist John Sykes, who pla..."
5,McAvoy and Lange set for Glasgow Film Festival,"Now in its 21st year the festival, which start...",https://www.bbc.com/news/articles/c79d3d38pp8o,feeds,articles,[BBC News],2025-01-21 06:10:29.415000+00:00,James McAvoy and Jessica Lange set for Glasgow...,X-Men star James McAvoy and double Oscar winne...
6,"Yes, Minister character is government's new AI...",A new suite of AI tools for civil servants are...,https://www.bbc.com/news/articles/cy48vl3p0nyo,feeds,articles,[BBC News],2025-01-21 00:05:26.507000+00:00,"Yes, Minister character is government's new AI...",Government workers will soon be given access t...
7,Extra £1.5m needed to complete Jersey Opera House,The theatre says additional funding is needed ...,https://www.bbc.com/news/articles/c1dgre230r3o,feeds,articles,[Elliot Ball],2025-01-21 06:08:17.025000+00:00,Extra £1.5m needed to complete Jersey Opera House,The Jersey Opera House has warned it must find...
8,Delay to TikTok ban gets Trump sign-off,The order was among a slew of directives Trump...,https://www.bbc.com/news/articles/cd0j24rj4ryo,feeds,articles,[Lily Jamali and Peter Hoskins],2025-01-21 02:12:14.130000+00:00,President Trump signs executive order delaying...,President Trump has signed an executive order ...
9,"Yes, Minister character is government's new AI...",A new suite of AI tools for civil servants are...,https://www.bbc.com/news/articles/cy48vl3p0nyo,feeds,articles,[BBC News],2025-01-21 00:05:26.507000+00:00,"Yes, Minister character is government's new AI...",Government workers will soon be given access t...
